In [0]:
# %% [code]
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# %% [code]
!java -version

# %% [code]
!ls;

# %% [code]
!wget -O solr-8.5.0.zip "https://archive.apache.org/dist/lucene/solr/8.5.0/solr-8.5.0.zip"

# %% [code]
!unzip solr-8.5.0.zip

# %% [code]
!ls

# %% [code]
!pip install pysolr
!pip install kazoo #Needed for python zookeeper

# %% [code]
!solr-8.5.0/bin/solr start -force

# %% [code]
!solr-8.5.0/bin/solr create -c covid19 -s 1 -rf 1 -force

# %% [code]
# Using _default configset with data driven schema functionality. NOT RECOMMENDED for production use.
!solr-8.5.0/bin/solr config -c covid19 -p 8983 -action set-user-property -property update.autoCreateFields -value false

# %% [code]
!curl -X POST -H 'Content-type:application/json' --data-binary '{"add-field-type" : {"name":"keywordText","class":"solr.TextField", "positionIncrementGap":"100", "indexAnalyzer" : {"tokenizer":{"class":"solr.KeywordTokenizerFactory" }, "filters":[{"class":"solr.TrimFilterFactory"},{"class":"solr.StopFilterFactory", "ignoreCase":true, "words":"lang/stopwords_en.txt"},{"class":"solr.ManagedSynonymGraphFilterFactory", "managed":"english" },{"class":"solr.RemoveDuplicatesTokenFilterFactory"},{"class":"solr.FlattenGraphFilterFactory"}]},"queryAnalyzer" : {"tokenizer":{"class":"solr.KeywordTokenizerFactory" },"filters":[{"class":"solr.TrimFilterFactory"},{"class":"solr.StopFilterFactory", "ignoreCase":true, "words":"lang/stopwords_en.txt"},{"class":"solr.ManagedSynonymGraphFilterFactory", "managed":"english" },{"class":"solr.RemoveDuplicatesTokenFilterFactory"}]}}}' http://localhost:8983/solr/covid19/schema

# %% [code]
!curl -X POST -H 'Content-type:application/json' --data-binary '{"add-field": {"name":"title", "type":"text_en_splitting", "multiValued":false, "stored":true, "indexed":true}}' http://localhost:8983/solr/covid19/schema;
!curl -X POST -H 'Content-type:application/json' --data-binary '{"add-field": {"name":"abstract", "type":"text_en_splitting", "multiValued":false, "stored":true, "indexed":true}}' http://localhost:8983/solr/covid19/schema;
!curl -X POST -H 'Content-type:application/json' --data-binary '{"add-field": {"name":"text_body", "type":"text_en_splitting", "multiValued":false, "stored":true, "indexed":true}}' http://localhost:8983/solr/covid19/schema;
!curl -X POST -H 'Content-type:application/json' --data-binary '{"add-field": {"name":"source", "type":"keywordText", "multiValued":false, "stored":true, "indexed":true}}' http://localhost:8983/solr/covid19/schema;

# %% [code]
import pysolr
solr = pysolr.Solr('http://localhost:8983/solr/covid19/', timeout=10)

# %% [code]
solr_content = {}
solr_content['id'] = 'id-9876'
solr_content['title'] = 'COVID-19 epidemic'
solr_content['source'] = 'Source-2'
solr_content['abstract'] = 'COVID-19 Epidemic causing tensions'
solr_content['text_body'] = 'Acute lung injury is a marked effect of the COVID-19 patients.'
solr.add([solr_content])
solr.commit()

# %% [code]
# Search for data
results = solr.search("\"acute lung injury\"", **{
    'hl': 'true',
    'hl.fragsize': 100,
    'hl.bs.type': 'SENTENCE',
    'hl.method' : 'unified',
    'hl.snippets' : 5,
    'df':"text_body"
})

print("Saw {0} result(s).".format(len(results)))

for result in results:
    print('TITLE : ', result['title'])
    #display(HTML('<h1>' + result['title'][0]) + '</h1>')
    hl_snippets = results.highlighting[result['id']]['text_body']
    
    for index, snippet in enumerate(hl_snippets, start=1):
        print('\t\t', index, '. ', snippet, '\n')
    print('############################')